In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
import torch 
import torch.nn.functional as F
from torch.autograd.functional import hessian
from torch.distributions.multivariate_normal import MultivariateNormal
import seaborn as sns

**Instruções gerais:** Sua submissão <u>deve</u> conter: 
1. Um "ipynb" com seu código e as soluções dos problemas
2. Uma versão pdf do ipynb

Favor <u>não</u> enviar um .zip dos arquivos.
Caso você opte por resolver as questões de "papel e caneta" em um editor de $\LaTeX$ externo, o inclua no final da versão pdf do 'ipynb'--- submetendo um **<u>*único pdf*</u>**.

# Trabalho de casa 03: Regressão logística e inferência Bayesiana aproximada

O pedaço de código abaixo carrega o banco de dados 'breast cancer' e adiciona uma coluna de bias. Além disse, ele o particiona em treino e teste.

1. Implemente a estimativa de máximo a posteriori para um modelo de regressão logística com priori $\mathcal{N}(0, c I)$ com $c=100$ usando esse banco de dados;
2. Implemente a aproximação de Laplace para o mesmo modelo;
3. Implemente uma aproximação variacional usando uma Gaussiana diagonal e o truque da reparametrização;
4. Calcule a accuracy no teste para todas as opções acima --- no caso das 2 últimas, a prob predita é $\int_\theta p(y|x, \theta) q(\theta)$;
5. Para cada uma das 3 técnicas, plote um gráfico com a distribuição das entropias para as predições corretas e erradas (separadamente), use a função kdeplot da biblioteca seaborn.
6. Comente os resultados, incluindo uma comparação dos gráficos das entropias.

Explique sua implementação também! 

Para facilitar sua vida: use PyTorch, Adam para otimizar (é uma variação SGD) com lr=0.001, use o banco de treino inteiro ao invés de minibatchces, use `binary_cross_entropy_with_logits` para implementar a -log verossimilhança, use `torch.autograd.functional` para calcular a Hessiana. Você pode usar as bibliotecas importadas na primeira célula à vontade. Verifique a documentação de `binary_cross_entropy_with_logits` para garantir que a sua priori está implementada corretamente, preservando as proporções devidas. Use 10000 amostras das aproximações para calcular suas predições.

In [19]:
data =  load_breast_cancer()
N = len(data.data)
Ntrain = int(np.ceil(N*0.6))
perm = np.random.permutation(len(data.data))
X = torch.tensor(data.data).float()
X = torch.cat((X, torch.ones((X.shape[0], 1))), axis=1) 
y = torch.tensor(data.target).float()

Xtrain, ytrain = X[perm[:Ntrain]], y[perm[:Ntrain]]
Xtest, ytest = X[perm[Ntrain:]], y[perm[Ntrain:]]

In [22]:
# Maximum a posteriori

class LogisticRegression(torch.nn.Module):
    def __init__(self, d: int):
        super(LogisticRegression, self).__init__()
        self.theta = torch.nn.Parameter(torch.empty(d).normal_(std=100 ** 0.5))
    
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return torch.sigmoid(self.linear(X))
    
def maximum_a_posteriori(X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    priori_var = 100
    learning_rate = 0.001

    d = X.shape[1]

    theta = torch.empty(d).normal_(std=priori_var ** 0.5).requires_grad_(True)
    optimizer = torch.optim.Adam([theta], lr=learning_rate)
    prior = MultivariateNormal(torch.zeros(d), priori_var * torch.eye(d))
    for _ in range(10000):
        optimizer.zero_grad()
        log_likelihood = F.binary_cross_entropy_with_logits(X @ theta, y)
        log_prior = -prior.log_prob(theta)
        loss = log_likelihood + log_prior
        loss.backward()
        print(loss.item())
        optimizer.step()
        if theta.grad.norm() < 1e-5:
            print("stop")
            break
        
    return theta
    

theta = maximum_a_posteriori(Xtrain, ytrain)

11017.1806640625
11016.10546875
11015.0263671875
11013.9501953125
11012.8720703125
11011.794921875
11010.7177734375
11009.6416015625
11008.5615234375
11007.4853515625
11006.4091796875
11005.33203125
11004.2529296875
11003.1767578125
11002.0986328125
11001.0224609375
10999.9453125
10998.8681640625
10997.7890625
10996.7138671875
10995.634765625
10994.5576171875
10993.48046875
10992.4033203125
10991.3251953125
10990.25
10989.1708984375
10988.0947265625
10987.017578125
10985.9404296875
10984.8623046875
10983.787109375
10982.70703125
10981.6298828125
10980.552734375
10979.4755859375
10978.3984375
10977.3212890625
10976.2431640625
10975.1669921875
10974.087890625
10973.01171875
10971.9326171875
10970.857421875
10969.779296875
10968.703125
10967.625
10966.5478515625
10965.470703125
10964.3935546875
10963.3154296875
10962.2392578125
10961.16015625
10960.0830078125
10959.005859375
10957.9287109375
10956.8515625
10955.7744140625
10954.6982421875
10953.6201171875
10952.5419921875
10951.4658203125

---

# Exercícios de "papel e caneta"

1. Derive a fórmula para a divergência KL entre duas distribuições Gaussianas univariadas, i.e., $D_\text{KL}\left(\mathcal{N}(\mu_1, \sigma_1^2) \| \mathcal{N}(\mu_2, \sigma_2^2)\right)$;

2. Suponha que $P$ é a família das distribuições categóricas com suporte em $\{1,\ldots, L\}$. Qual $p \in P$ possui maior entropia? 

3. Use a [desigualdade de Jensen](https://en.wikipedia.org/wiki/Jensen%27s_inequality) para mostrar que a divergência KL é não-negativa.

> **Dica:** A desigualdade de Jensen afirma que, se $\varphi$ é uma função convexa, então $\varphi(\mathbb{E}[X]) \leq \mathbb{E}[\varphi(X)]$.

4. Derive a aproximação de Laplace para a distribuição [Beta](https://en.wikipedia.org/wiki/Beta_distribution)($\alpha, \beta$). Mostre uma fórmula para valores genéricos $\alpha,\beta>1$ e a instancie para $\alpha=\beta=2$.

5. Derive a posteriori para o modelo Bayesiano com verossimilhança [Categórica](https://en.wikipedia.org/wiki/Categorical_distribution) e priori [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution), i.e.:
$$
\begin{align}{2}
y_1,\ldots, y_N &\sim Cat(\mathbf{\theta})\\
\mathbf{\theta} &\sim Dirichlet(\mathbf{\alpha})
\end{align}
$$
onde $\mathbf{\theta}$ e $\mathbf{\alpha}$ são vetores $L$-dimensionais.


### Questão 
$$D_\text{KL}\left(\mathcal{N}(\mu_1, \sigma_1^2) \| \mathcal{N}(\mu_2, \sigma_2^2)\right)$$
